In [1]:
import sys
sys.path.append('.')

In [2]:
import yaml
import numpy as np
import pandas as pd

In [3]:
with open('configs/test.yaml', 'r') as f:
    config = yaml.load(f, yaml.Loader)
config

{'warehouse': {'data_path': 'input/',
  'train': True,
  'create_args': {'use_geo': True,
   'use_era5': True,
   'era5_metrics': '*',
   'era5_years': '*'}},
 'featurise': {'calcers': {'dates_features': {},
   'geo_cat_features': {},
   'geo_neighbors_features': {'count_neighbors': 5},
   'grib_features': {'metrics': '*',
    'pooling_size': 3,
    'lags': [0, 1, 2, 3],
    'agg_funcs': ['max', 'mean']},
   'target_base': {}}},
 'pipeline': {'name': 'baseline_pipeline',
  'estimators': {'transformes': [],
   'selectors': [{'dummy_selector': {}}],
   'model': {'cat_boost_classifier': {}}},
  'split': {},
  'search': {}}}

In [4]:
from competition import Repository

In [5]:
repo = Repository()
repo._objects

{'dates_features': competition.featurise.calcers.DatesFeaturesCalcer,
 'geo_cat_features': competition.featurise.calcers.GeoCatFeaturesCalcer,
 'geo_neighbors_features': competition.featurise.calcers.GeoNeighborsFeaturesCalcer,
 'grib_features': competition.featurise.calcers.GribFeaturesCalcer,
 'target_base': competition.featurise.calcers.TargetBaseCalcer,
 'dummy_selector': competition.estimators.selectors.DummySelector,
 'baseline_pipeline': competition.pipeline.pipelines.BaselinePipeline}

# Pipeline

In [4]:
features_df = pd.read_csv('features.csv')
features_df.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,dt,grid_index,month,week,day_of_week,geo_population,geo_place,geo_cn5_city_mean_distance,geo_cn5_city_max_distance,geo_cn5_city_min_distance,...,wind_tp_ws3_mean_lag_3,infire_day_1,infire_day_2,infire_day_3,infire_day_4,infire_day_5,infire_day_6,infire_day_7,infire_day_8,infire_day_num
0,2020-05-04,143,5,19,0,NaN,NaN,0.344685,0.378005,0.288359,...,2.064614e-04,0,0,0,0,0,0,0,0,0
1,2021-02-24,891,2,8,2,NaN,NaN,0.290611,0.404587,0.164580,...,1.102550e-03,0,0,0,0,0,0,0,0,0
2,2021-02-27,891,2,8,5,NaN,NaN,0.290611,0.404587,0.164580,...,3.826576e-03,0,0,0,0,0,0,0,0,0
3,2021-04-01,892,4,13,3,2054.0,village,0.216737,0.292387,0.096216,...,9.598749e-06,0,0,0,0,0,0,0,0,0
4,2020-03-14,893,3,11,5,NaN,village,0.189796,0.267262,0.103796,...,8.523463e-07,0,0,0,0,0,0,0,0,0


# Collect features

In [3]:
from competition.warehouse import Warehouse
from competition.featurise import Featuriser

In [5]:
engine = (Warehouse(data_path=config['warehouse']['data_path'],
                    train=config['warehouse']['train'])
          .create(config['warehouse']['create_args']))
engine

In [6]:
features_df = Featuriser(engine).get_features(config['featurise']['calcers'])
features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 488103 entries, 0 to 488102
Columns: 219 entries, dt to infire_day_num
dtypes: float32(200), float64(3), int64(13), object(3)
memory usage: 446.9+ MB


In [7]:
features_df.columns

Index(['dt', 'grid_index', 'month', 'week', 'day_of_week', 'geo_population',
       'geo_place', 'geo_cn5_city_mean_distance', 'geo_cn5_city_max_distance',
       'geo_cn5_city_min_distance',
       ...
       'wind_tp_ws3_mean_lag_3', 'infire_day_1', 'infire_day_2',
       'infire_day_3', 'infire_day_4', 'infire_day_5', 'infire_day_6',
       'infire_day_7', 'infire_day_8', 'infire_day_num'],
      dtype='object', length=219)

In [8]:
features_df.head()

,dt,grid_index,month,week,day_of_week,geo_population,geo_place,geo_cn5_city_mean_distance,geo_cn5_city_max_distance,geo_cn5_city_min_distance,...,wind_tp_ws3_mean_lag_3,infire_day_1,infire_day_2,infire_day_3,infire_day_4,infire_day_5,infire_day_6,infire_day_7,infire_day_8,infire_day_num
0,2020-05-04,143,5,19,0,NaN,NaN,0.344685,0.378005,0.288359,...,2.064614e-04,0,0,0,0,0,0,0,0,0
1,2021-02-24,891,2,8,2,NaN,NaN,0.290611,0.404587,0.164580,...,1.102550e-03,0,0,0,0,0,0,0,0,0
2,2021-02-27,891,2,8,5,NaN,NaN,0.290611,0.404587,0.164580,...,3.826576e-03,0,0,0,0,0,0,0,0,0
3,2021-04-01,892,4,13,3,2054,village,0.216737,0.292387,0.096216,...,9.598749e-06,0,0,0,0,0,0,0,0,0
4,2020-03-14,893,3,11,5,None,village,0.189796,0.267262,0.103796,...,8.523463e-07,0,0,0,0,0,0,0,0,0


In [11]:
features_df.describe(include='all').T.tail(20)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
wind_v10_ws3_mean_lag_2,488024,NaN,NaN,NaN,0.155824,2.26971,-15.2033,-1.30447,0.147992,1.57001,19.2576
wind_tp_ws3_max_lag_2,488024,NaN,NaN,NaN,0.000941589,0.00215276,0,1.34669e-06,0.000117246,0.000844103,0.0952358
wind_tp_ws3_mean_lag_2,488024,NaN,NaN,NaN,0.000707508,0.00173172,0,8.80141e-07,6.79918e-05,0.000575052,0.0819341
wind_sp_ws3_max_lag_3,488024,NaN,NaN,NaN,98350.5,3389.2,73235.9,97092.4,99357.7,100609,105527
wind_sp_ws3_mean_lag_3,488024,NaN,NaN,NaN,97586.9,3950.81,72082.9,96087.4,98856.8,100252,105359
wind_u10_ws3_max_lag_3,488024,NaN,NaN,NaN,0.732242,2.36844,-16.4305,-0.752289,0.801353,2.05615,19.3251
wind_u10_ws3_mean_lag_3,488024,NaN,NaN,NaN,0.371752,2.35305,-17.1395,-1.10282,0.400017,1.76687,17.9639
wind_v10_ws3_max_lag_3,488024,NaN,NaN,NaN,0.549664,2.29223,-15.6073,-0.962599,0.611069,1.90231,19.319
wind_v10_ws3_mean_lag_3,488024,NaN,NaN,NaN,0.179181,2.27675,-16.0265,-1.29002,0.169922,1.59368,18.8487
wind_tp_ws3_max_lag_3,488024,NaN,NaN,NaN,0.00096009,0.00217655,0,1.70469e-06,0.000123225,0.000864309,0.111974


In [12]:
features_df.to_csv('faetures.csv', index=False)